### Attributes defination:
1. Subject (IoT device or user) attributes
- role: employee, visitor, security staff, admin
- department: HR, Engineering, Maintenance
- securityLevel: 1–5
- deviceType: sensor, actuator, gateway
- location: floor number or zone

2. Object (resource/device) attributes
- resourceType: temperatureSensor, doorLock, camera, HVAC
- sensitivityLevel: low, medium, high
- location: same zone/floor

3. Environmental attributes
- time: working hours or not
- alertState: normal, emergency, lockdown
- motionDetected: yes/no

### Policy defination


---
Registration phase:

We use elliptic curve encryption to ensure security communication. 

Step 1 — Registration Request by IoT Device  
  IoT sends basic identity info:  
 - deviceID
 - manufacturerID
 - model
 - firmwareVersion
 - MAC / serial  
These are claims, not trusted yet.

Step 2 — Secure Channel Establishment  
IoT encrypts registration data using fog TEE public key:
 - C = Enc(PK_fogTEE, registration_data)  
Only TEE can decrypt → protects against compromised fog OS.

Step 3 — Evidence Verification Inside Fog TEE  
Fog TEE verifies: 
 - Manufacturer signature on firmware
 - Certificate chain validity
 - Device uniqueness
 - Network consistency  
If verification fails → registration rejected.

Step 4 — Attribute Authority Decision Logic  
The AA (logically executed in fog TEE or coordinated AAs) assigns attributes based on verified evidence:  
Example decision table:
| Evidence                    | Assigned Attribute        |
| --------------------------- | ------------------------- |
| Verified firmware signature | `deviceType = sensor`     |
| Admin deployment record     | `ownerDomain = FactoryA`  |
| Fog cluster location        | `location = FogCluster03` |
| IDS benign behavior         | `securityLevel = 0.75`    |
  
Attributes are never inferred from device claims alone.

Step 5 - Attribute Blinding  
AA bind the set of attributes with deviceID then sign it to proove certification.  

Step 6 Storage  
Attributes and access policies are encrypted using a symmetric key protected by the fog node’s TEE and stored outside the enclave on the fog node. To prevent data loss caused by fog node failures, the fog node periodically backs up authorization state to a trusted cloud-based DDS.

During backup, the TEE decrypts the local data, attaches metadata including owner identity, version number, and timestamp, and transmits the data to the cloud together with a TEE-generated signature. Upon receipt, the cloud verifies the signature and freshness before storing the backup data. This design enables efficient recovery while preserving integrity and authenticity of authorization information.

To ensure availability and fault tolerance, the fog node periodically backs up encrypted attributes and access policies to a trusted cloud-based DDS. The backup process is executed within the fog node’s TEE, which decrypts locally stored data using a sealed symmetric key, attaches metadata including owner identity, version number, and timestamp, and encrypts the backup payload for the cloud. The TEE further signs the encrypted payload to guarantee authenticity and integrity. Upon receiving the backup, the cloud verifies the signature and freshness before storing the data, enabling secure and efficient recovery in the event of fog node failure.

---
Upon receiving an access request, the fog node first authenticates the requesting IoT device by verifying its digital signature and checks request freshness using timestamps or nonces to prevent replay attacks. The fog node then verifies the existence and validity of the requested resource and its associated access policy before proceeding. Once these preliminary checks are completed, the fog node forwards the encrypted subject attributes and resource access policy to the Trusted Execution Environment (TEE) for authorization.

Within the enclave, the TEE securely decrypts the protected attributes and policies using enclave-bound keys and evaluates the access request according to predefined Attribute-Based Access Control (ABAC) rules. Because all sensitive authorization logic and data processing are confined within the enclave, neither a compromised fog operating system nor malicious insiders can manipulate attributes, alter policies, or influence authorization outcomes. The resulting access decision is then returned to the fog node for enforcement, enabling secure, fine-grained, and tamper-resistant authorization at the network edge